In [1]:
import numpy as np
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO import history as hs
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

D:\GitProjects\Bio-comp\Coursework\ANNModel\model.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Cubic (10 runs) mean fitness: 252
Cubic_pso_config_mean = [65.07982407, 4.95690714, 0.28949701, 0.54421663, 0.39890445, 1.42315492, 1.18854163]

# Linear (10 runs) mean fintess: 9.17486598344206e+24
Linear_pso_config_mean = [75.13534983, 6.59595661, 0.15486748, 0.63274028, 0.93549642, 1.5667033, 0.48415203]

# Tanh (10 runs) mean fitness: 23659 
Tanh_pso_config_mean = [54.98334367, 6.41212707, 0.59507185, 1.23273439, 1.68258425, 1.01222575, 0.19820747]

# Sine (10 runs) mean fitness: 71
Sine_pso_config_mean = [72.77234021, 6.72388576, 0.60373995, 0.38615018, 1.022057, 0.86230487, 0.59935222]

# Complex (10 runs) mean fitness: 13
Complex_pso_config_mean = [8.99258905e+01, 5.57111071e+00, 5.69030292e-01, 3.38835134e-02, 1.24389435e+00, 1.35151796e+00, 4.87958932e-01]

# Xor (10 runs) mean fitness 873815265613870.6
Xor_pso_config_mean = [8.64423020e+01, 6.33181111e+00, 7.82660047e-02, 1.43692310e+00, 1.70649966e+00, 1.06783274e+00, 5.57344119e-01]


In [3]:
# Cubic Most fit model: 4436
Cubic_ann_params = np.array([ 4.06948631e+00, -7.29125756e-01,  5.73721712e-01, -4.97778569e-01, 1.21867021e-01, -4.07830431e-01,  9.23118308e-01,  9.96763804e-01, -3.53176479e-01,  2.82162360e+00, -6.77411971e-01, -6.33183382e-01, 3.17935306e-01, -1.91733120e-01,  2.57710600e-03])

# Linear Most fit model: 9.17486598344206e+25
Linear_ann_params = np.array([0.33653903, 0.62395429, 0.38036724,-0.73197691, 0.27957089, -0.44447831, 0.90125204, -0.72400064, -0.73325263, 0.2053941, -0.37270846, 0.1568939, 0.4157871, -0.37861424, -0.57400409])

# Tanh Most fit model: 236154
Tanh_ann_params = np.array([4.30993349, 0.2291253, -0.99626464, -0.29776704,  0.99032436, 0.54519926, 0.98178641, -0.06046016, 0.9675356, 2.17890815, -0.49269196, -0.29330431, -0.94045692, 0.72556016, 0.95398864])

# Sine Most fit model: 488
Sine_ann_params = np.array([ 4.34214646, -0.37620669, -0.1737171 ,  0.98924977,  0.94187429, -0.35848436,  0.03053108,  0.31920693, -0.32693426,  2.7825707, -0.7316042, -0.33840137, -0.59207658, -0.77175905, 0.91605957])

# Complex Most fit model: 
Complex_ann_params = np.array([ 3.90353086, 0.38052382, -0.55386827, 0.08622513, -0.44906989, 0.61901606, -0.87650059, -0.625452, -0.98830069, 0.85692286, 0.01958315, -0.2346267, 0.35945306, 1.73135609, -0.28885139, 0.0893994 , -0.47053919,  0.00421956, 0.40903142])

# Xor Most fit model: 8738152656138548
Xor_ann_params = np.array([ 4.02523790e+00,  6.23414094e-01,  9.76014400e-03,  2.37501907e-01, 1.61720158e-01, -4.58494817e-01,  5.43701583e-01,  1.31984645e-03, 4.74499415e-01, -8.82859850e-01, -1.29200272e-01, -3.78243447e-01, -9.62890568e-01,  3.54267095e-01, -4.41831336e-01, -9.22734200e-01, 6.87573835e-01,  3.54729391e-01,  7.20793804e-01])

In [4]:
data = d.Data("./Data/1in_cubic.txt")
decimals = 'cubic'
X = data.get_rows()
y = data.get_output()

psoParams = Xor_pso_config_mean
annParams = Xor_ann_params

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

In [5]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = hs.PSOHistory(model)

Model Compiled!


In [6]:
pso = ps.PSO(swarm_size=90, termination_args={'max_iter': 400}, verbose=True)
#pso = pso.decode_vec(psoParams)
print('Sanity check swarm size: ',pso.swarm_size)

Sanity check swarm size:  90


### Run experiment with the PSO hyperparameters calculated by meta-PSO 
#### This will find the 10 most fit ANN parameters, produce a mean accuracy*, Fitness, and loss for both test and train.

*This accuracy is calculated with an absolute tolerance of 0.03 for all tests

In [7]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)

train ={
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

test = {
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

for i in range(10):
    fitness = pso.run()
    # Evaluating on training set
    model.set_training_input(X_train, y_train)
    train['fitnessList'].append(model.evaluate_fitness(fitness.location))
    score = data.score(model, decimals, atol=0.03)
    train['scoreA'].append(score[0])
    train['scoreB'].append(score[1])
    train['lossList'].append(model.loss)
    # Evaluating on testing set
    model.set_training_input(X_test, y_test)
    test['fitnessList'].append(model.evaluate_fitness(fitness.location))
    score = data.score(model, decimals, atol=0.03)
    test['scoreA'].append(score[0])
    test['scoreB'].append(score[1])
    test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))


Testing on train - A:  0.1253731343283582  B:  0.1253731343283582  Fitness:  23.059  Loss:  0.049
Testing on test - A:  0.11764705882352941  B:  0.11764705882352941  Fitness:  29.477  Loss:  0.039


In [8]:
#index = np.argmax(model_history.vec_history)
#best_ANN_params_new = model_history.vec_history[index]
#print(best_ANN_params_new)

In [9]:
#pso.set_search_dimensions(model.dimension_vec())
#pso.set_fitness_fn(model_history.evaluate_fitness)
#fitness = pso.run()
#print(fitness.location)
print(fitness.fitness)

11.66413304157858


In [10]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.RELU
bias:  [0.08587748]
weights:  [[-0.6718145 ]
 [ 0.1346022 ]
 [-0.10738429]
 [ 0.58566799]]


In [11]:
model.decode_vec(annParams)
model.one_pass()

train ={
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

test = {
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

# Evaluating on training set
model.set_training_input(X_train, y_train)
train['fitnessList'].append(model.evaluate_fitness(annParams))
score = data.score(model, decimals, atol=0.03)
train['scoreA'].append(score[0])
train['scoreB'].append(score[1])
train['lossList'].append(model.loss)
# Evaluating on testing set
model.set_training_input(X_test, y_test)
test['fitnessList'].append(model.evaluate_fitness(annParams))
score = data.score(model, decimals, atol=0.03)
test['scoreA'].append(score[0])
test['scoreB'].append(score[1])
test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))


ValueError: -1 is not a valid ActivationFunction

In [ ]:
print(model.y, model.y_hat)

In [ ]:
1/model.loss

In [ ]:
model.evaluate_fitness(annParams)